In [ ]:
import pandas as pd
import os
import sys
sys.path.insert(0, os.path.abspath('H:/code/gs_quant'))
sys.path.insert(0, os.path.abspath('H:/code/gs_quant_analytics'))
sys.path.insert(0, os.path.abspath('H:/code/visual-structuring-core'))
sys.path.insert(0, os.path.abspath('H:/code/gs_quant_internal'))
sys.path.insert(0, os.path.abspath('H:/code/inventa-rates-utils'))
sys.path.insert(0, os.path.abspath('H:/code/franchise-rnd'))
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from gs_quant.session import GsSession

GsSession.use()

In [ ]:
from gs_quant.session import Environment
from gs_quant.instrument import FXOption
from gs_quant.backtests.strategy import Strategy
from gs_quant.backtests.triggers import *
from gs_quant.backtests.actions import *
from gs_quant.backtests.equity_vol_engine import *
from gs_quant.backtests.generic_engine import GenericEngine
from gs_quant.risk import Price
from datetime import datetime, date

import gs_quant.risk as risk

In [ ]:
# Initialize session
from gs_quant.session import GsSession
GsSession.use(client_id=None, client_secret=None, scopes=('run_analytics',)) 

#### If we have a periodic trigger which triggers every 1w and has an add trade action which has a trade duration of 1w sometimes you can end up with a situation where you have 2 trades or 0 trades.  
#### This is because the schedule of 1w dates for the trigger will derive a schedule of 1w dates and then adjust each date for holidays.  Where as if you take a specific date and add a week this will not take note of the holiday and therefore may give a different date.

In [ ]:
# Define backtest dates
start_date = date(2024, 3, 24)
end_date = date(2024, 5, 1)

In [ ]:
call = FXOption(pair='EURUSD', expiration_date='3m', option_type='Call', name='call')

### Entering a position weekly

In [ ]:
# To demonstrate create add trade action with a duration of 1w on a periodic trigger of 1w across a holiday
hol_cal = (dt.date(2024, 3, 29), dt.date(2024, 4, 1))
trade_action = AddTradeAction(priceables=call, trade_duration='1w', name='weekly_duration', 
                              holiday_calendar=hol_cal)
trade_trigger = PeriodicTrigger(trigger_requirements=PeriodicTriggerRequirements(start_date=start_date, 
                                                                                 end_date=end_date, frequency='1w', 
                                                                                 calendar=hol_cal),
                                actions=trade_action)

strategy = Strategy(None, trade_trigger)

# Run backtest daily
GE = GenericEngine()
backtest = GE.run_backtest(strategy, start=start_date, end=end_date, frequency='1b', show_progress=True)

In [ ]:
# View backtest trade ledger
backtest.trade_ledger()

#### note that the second trade closes on the 4th April but the third trade opens on the 5th.

In order to fix this we can set the trade duration to <span style="color:blue">"next schedule"</span>.
This will have the effect of forcing the close date to line up with the open date of the following trade.

This should only be used if you want the trade duration of the trade and the frequency of the trigger to line up 
and the frequency is not daily.  Thereby chaining (or rolling) the positions.

In [ ]:
hol_cal = (dt.date(2024, 3, 29), dt.date(2024, 4, 1))
trade_action = AddTradeAction(priceables=call, trade_duration='next schedule', name='weekly_duration', holiday_calendar=hol_cal)
trade_trigger = PeriodicTrigger(trigger_requirements=PeriodicTriggerRequirements(start_date=start_date, end_date=end_date, frequency='1w', 
                                                                                 calendar=hol_cal),
                                actions=trade_action)

strategy = Strategy(None, trade_trigger)

# Run backtest daily
GE = GenericEngine()
backtest = GE.run_backtest(strategy, start=start_date, end=end_date, frequency='1b', show_progress=True)

In [ ]:
# View backtest trade ledger
backtest.trade_ledger()

#### now note that all the trades start and end dates line up